# Networks and their Structure Assignment

## Network Science Topics 2 and 4

Note that the networks in this exercise are undirected.

Recall the Watts-Strogatz network model from Topic 2.  Here we define a similar model which we call the **Variable Degree Watts-Strogatz (VDWS) model**.  This is how a **VDWS-network** is constructed.

* Let $n$ and $m$ be positive integers.  Let $p$ be a real number between 0 and 1.  Create a set of $n$ vertices labelled 0 to $n-1$.  We think of these as being arranged on a circle so if two adjacent vertices are close to each other on the circle, it is clear what is meant if we say that one is the clockwise neighbour of the other.  For example 4 is a clockwise neighbour of 2.
* For each $v$, let $\ell(v)$ be the local degree of $v$.  Choose the values $\ell(v)$ randomly such that they have a zero-truncated Poisson distribution with parameter $m$ (see below).  
* Join each vertex $v$ to vertices $v - \ell(v), v - \ell(v) + 1, \ldots, v-1, v+1, \ldots v + \ell(v) - 1, v + \ell(v)$ where addition is $\bmod n$.  Thus each vertex will have degree at least $2\ell(v)$, but it might be more.  For example, if 3 has local degree 2, then it will be adjacent to 1, 2, 4 and 5.   But if 10 has local degree 7, there will also be an edge between 3 and 10.   
* For each vertex $v$, for each edge from $v$ to a clockwise neighbour $w$: with probability $p$, the edge from $v$ to $w$ is deleted and replaced by an edge from $v$ to a vertex $x$ chosen uniformly at random from all the vertices in the network. We call this process *rewiring*. If $x=v$ or $x$ is already a neighbour of $v$, then we do nothing and the edge from $v$ to $w$ is kept.  (Note that in this way, each of the edges created in the previous step should be considered exactly once for rewiring, and that edges that are created by rewiring should not be later rewired themselves.)

You will need to write code to create VDWS-networks.  You can use the code for WS-networks (see Topic 4 on Learn Ultra) as a starting point.  To create values that are sampled from a zero-truncated Poisson distribution with parameter $m=10$, say, you can use the following code.

```python
import numpy as np
local_degrees = np.random.poisson(10, 100)
local_degrees = local_degrees[local_degrees > 0]
```

Note that the number of values created cannot be predicted precisely since the second line samples 100 values from a Poisson distribution and then the third line removes the zeros (although zeros are very unlikely for the value of $m$ we will use).

1. [20 marks] Consider the epidemic model with vaccinations from the Topic 4 lecture notes with states S, I, V, VI and R.  Using this model, simulate the spread of disease on a VDWS-network with $n=200000$, $m=25$ and $p=0.01$.  Assume that initially 5 randomly chosen vertices are in I and every other node is in S.  From $t=50$, at each time step move 400 randomly chosen vertices from S to V (until S is empty).  Let $t_I=2$ and consider various values for the other parameters.   Initially let $p(\mbox{I}, \mbox{S})=p(\mbox{I}, \mbox{V})=p(\mbox{VI}, \mbox{S})=p(\mbox{VI}, \mbox{V})=0.01$ (implying that the vaccination is ineffective).   Then consider cases where $p(\mbox{I}, \mbox{S})=0.01$ but the other probabilities are lower, modelling the cases where the vaccination protects against infection, transmission or both.  In each case, create plots that show  how the number of vertices in each of the five states varies over time.  Comment on your findings.

2. [25 marks] Repeat the simulations of the previous question, but with the single change that the 400 vertices moved from S to V at each time step (after $t=50$) can be strategically chosen (rather than being chosen at random).  Propose and test three different strategies and comment on their effectiveness.

With the values given, each simulation should require no more than 400 time steps (before the infection dies out).  You should be able to run this in a few minutes on your own computer (or any university PC).  If you find that with the values I have given, the simulations take too long, then choose alternative values and document this in your submission.

In [25]:
from typing import Dict, Set, List, Literal
import numpy as np
import random

Node = Vertex = int
Network = Graph = Dict[Node, Set[Node]]
NodeState = Literal["S", "I", "V", "VI", "R"]

In [26]:
def vdws_network(n_: int, m_: int, p_: float) -> Network:
    network_: Network = {i_: set() for i_ in range(n_)}

    local_degrees_: np.array = np.zeros(0)
    while len(local_degrees_) != n_:
        local_degrees_ = np.random.poisson(m_, n_)
        local_degrees_ = local_degrees_[local_degrees_ > 0]

    # Generate circular edges from local degrees
    for node_ in range(n_):
        local_degree_ = local_degrees_[node_]
        for neighbour_ in range(node_ - local_degree_, node_ + local_degree_ + 1):
            neighbour_ = neighbour_ % n_
            if node_ == neighbour_:
                continue

            network_[node_].add(neighbour_)
            network_[neighbour_].add(node_)

    # Rewire network
    for node_ in range(n_):
        local_degree_ = local_degrees_[node_]
        for clockwise_neighbour_ in range(node_, node_ + local_degree_ + 1):
            clockwise_neighbour_ = clockwise_neighbour_ % n_
            if random.random() < p_:
                new_neighbour_ = random.randint(0, n_ - 1)
                if new_neighbour_ != node_ and new_neighbour_ not in network_[node_]:
                    if clockwise_neighbour_ in network_[node_]:
                        network_[node_].remove(clockwise_neighbour_)
                    if node_ in network_[clockwise_neighbour_]:
                        network_[clockwise_neighbour_].remove(node_)

                    network_[node_].add(new_neighbour_)
                    network_[new_neighbour_].add(node_)

    return network_

In [28]:
vdws_network(10, 1, 0.5)

{0: {2, 3, 4, 7, 8, 9},
 1: {5, 7},
 2: {0, 3, 4, 5, 6, 9},
 3: {0, 2, 4, 5},
 4: {0, 2, 3, 5, 6},
 5: {1, 2, 3, 4, 7},
 6: {2, 4},
 7: {0, 1, 5, 8},
 8: {0, 7, 9},
 9: {0, 2, 8}}

In [36]:
def epidemic(
        network_: Network,
        p_i_s: float,
        p_i_v: float,
        p_vi_s: float,
        p_vi_v: float,
        vaccinations_per_tick: int = 400,
        ti: int = 2
) -> None:
    n_ = len(network_)
    node_states_: List[NodeState] = ["S"] * n_
    s_nodes_: Set[Node] = set(network_)
    i_nodes_: Dict[Node, int] = {}
    v_nodes_: Set[Node] = set()
    vi_nodes_: Dict[Node, int] = {}
    r_nodes_: Set[Node] = set()

    # Initial infection
    for i_node_ in random.sample(list(network_), 5):
        s_nodes_.remove(i_node_)
        node_states_[i_node_] = "I"
        i_nodes_[i_node_] = ti

    # Main simulation loop
    t_ = 0
    while True:
        # Vaccination uptake simulation
        for v_node_ in set(random.sample(s_nodes_, min(vaccinations_per_tick, len(s_nodes_)))):
            s_nodes_.remove(v_node_)
            v_nodes_.add(v_node_)
            node_states_[v_node_] = "V"

        # Simulate all the infected nodes
        for i_node_ in set(i_nodes_):
            i_nodes_[i_node_] -= 1

            # Simulate neighbour infection
            for neighbour_ in network_[i_node_]:
                # Simulate infection if susceptible
                if node_states_[neighbour_] == "S" and random.random() < p_i_s:
                    s_nodes_.remove(neighbour_)
                    i_nodes_[neighbour_] = ti
                    node_states_[neighbour_] = "I"

                # Simulate infection if vaccinated
                if node_states_[neighbour_] == "V" and random.random() < p_i_v:
                    v_nodes_.remove(neighbour_)
                    vi_nodes_[neighbour_] = ti
                    node_states_[neighbour_] = "VI"

            # Remove those who have been infected for ti ticks
            if i_nodes_[i_node_] == 0:
                i_nodes_.pop(i_node_)
                node_states_[i_node_] = "R"

        # Simulate all the infected vaccinated nodes
        for vi_node_ in set(vi_nodes_):
            vi_nodes_[vi_node_] -= 1

            # Simulate neighbour infection
            for neighbour_ in network_[vi_node_]:
                # Simulate infection if susceptible
                if node_states_[neighbour_] == "S" and random.random() < p_vi_s:
                    s_nodes_.remove(neighbour_)
                    i_nodes_[neighbour_] = ti
                    node_states_[neighbour_] = "I"

                # Simulate infection if vaccinated
                if node_states_[neighbour_] == "V" and random.random() < p_vi_v:
                    v_nodes_.remove(neighbour_)
                    vi_nodes_[neighbour_] = ti
                    node_states_[neighbour_] = "VI"

            # Remove those who have been infected for ti ticks
            if vi_nodes_[vi_node_] == 0:
                vi_nodes_.pop(vi_node_)
                node_states_[vi_node_] = "R"

        t_ += 1

        if t_ > 1000:
            break

In [30]:
network = vdws_network(200000, 25, 0.01)

In [37]:
epidemic(
    network,
    0.01,
    0.01,
    0.01,
    0.01,
    400,
    2
)